# Introduction to GMSO's base data structures

In this notebook, we will explore the base data strucures, pythonic features and design decisions for the `gmso` package.

Particularly, we will cover the following aspects of the package:

1. The abstract base classes module
2. Extension of abc: Site vs Atom: Example implementing a new site
3. Core Classes:
    - Sites
    - Connections
    - Potentials
    - Topology
    - ForceField
4. XML Schema for GMSO Forefield (focus on unyts(yt.unyt) and functional form (sympy)
5. Limitations, future plans

## Module gmso.abc
This module provides the abstract base classes for all other core data structures used in gmso. Our abstract base classes inherit from [pydantic](https://pydantic-docs.helpmanual.io/)'s `BaseModel` class which provides type hints as well as runtime data validation together with out-of-the-box serialization. The module structure is as follows:
```
gmso/abc 
├── abstract_connection.py 
├── abstract_potential.py 
├── abstract_site.py 
├── gmso_base.py 
```


1. [`gmso_base.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/gmso_base.py): Defines the class `GMSOBase` i.e. The base class for all our other classes that tweaks pydantic's `BaseModel` class to provide an ID based hasing as well as inject's numpydoc style docstrings from the fields of the class.

1. [`abstract_site.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_site.py): Defines the `Site` class which provides a plain topology site with following features: (a.) name (b.) position (c.) label

1. [`abstract_potential.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_potential.py): Defines the `AbstractPotential` class which is the base class for our `ParametricPotentials` as well as `PotentialTemplates`.

1. [`abstract_connection.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_site.py): Defines the `AbstractConnection` class which is the base class for our `Bond`, `Angle`, `Dihedral` and `Improper` classes.

## Example Implementation: Bead

The `Bead` class can now be implemented as a subclass of the abstract `Site` class. We can use the existing attributes from the super class like `name`, `position` etc... and define new attributes and methods for `Bead`. The goal is the consolidation of as many universal characteristics of a generic topology site into a base class (`Site`) and tweak its down-stream usage according to the needs of a particular site(like an `Atom` or a `Bead`). Usage of `Site` to create an `Bead` class is shown below:

In [ ]:
import unyt as u
from pydantic import Field, val

from gmso.abc.abstract_site import Site


class Bead(Site):
    __base_doc__ = "Basic Bead class inheriting from the Site Class"
    

my_bead = Bead()
my_bead.name  # When you inherit, the attribute(field) `name` is injected as the class name(Bead in this case)

In [20]:
# Docuementation is injected automatically as well
%pdoc Bead

## Core Classes
In `gmso` we define the following core classes. All of our core classes make use of `gmso.abc` to define a particular site, connection or potential. The module `gmso.core`'s structure is as follows:

```
gmso/core/
├── angle.py
├── angle_type.py
├── atom.py
├── atom_type.py
├── bond.py
├── bond_type.py
├── box.py
├── dihedral.py
├── dihedral_type.py
├── element.py
├── forcefield.py
├── improper.py
├── improper_type.py
├── parametric_potential.py
└── topology.py
```


### Sites
1. [`atom.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/atom.py): Defines the class `gmso.core.atom.Atom` which inherits from `gmso.abc.abstract_site.Site` to define an `Atom`.


### Connections
1. [`angle.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/angle.py): Defines the class `gmso.core.angle.Angle` which inherits from `gmso.abc.abstract_connection.Connection` to define a 3-partner connection between `Atoms`.

2. [`bond.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/bond.py): Defines the class `gmso.core.bond.Bond` which inherits from `gmso.abc.abstract_connection.Connection` to define a 2-partner connection between `Atoms`.

3. [`dihedral.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/dihedral.py): Defines the class `gmso.core.dihedral.Dihedral` which inherits from `gmso.abc.abstract_connection.Connection` to define a 4-partner connection(dihedral) between `Atoms`.

4. [`improper.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/improper.py): Defines the class `gmso.core.improper.Improper` which inherits from `gmso.abc.abstract_connection.Connection` to define a 4-partner connection(improper) between `Atoms`.

### Potentials
1. [`parametric_potential.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/parametric_potential.py): Defines the class `gmso.core.parametric_potential.ParametricPotential` which inherits from `gmso.abc.abstract_potential.Potential` to define a `ParametricPotential` class which stores the functional form of a Potential as sympy expression and parameters of the potential expression as `unyt_quantities`.

2. [`atom_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/atom_type.py): Defines the class `gmso.core.atom_type.AtomType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for an `AtomType`.

3. [`bond_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/bond_type.py): Defines the class `gmso.core.bond_type.BondType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for a `BondType`.

4. [`angle_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/angle_type.py): Defines the class `gmso.core.angle_type.AngleType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for an `AngleType`.

5. [`dihedral_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/dihedral_type.py) and [`improper_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/atom_type.py): Defines the classes `gmso.core.atom_type.DihedralType` and `gmso.core.improper_type.ImproperType` which inherit from `gmso.core.parametric_potential.ParametricPotential` which describe properties for a `DihedralType` and `ImproperType` respectively.

### Topologies
1. [`topology.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/topology.py): Defines the class `gmso.core.topology.Topology` which is the main data structure responsible for interactions between various potentials, connections and sites to form a chemical topology representation.

### Forcefield
1. [`forcefield.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/forcefield.py): Defines the class `ForceField` which defines the in memory representation of the gmso forcefield schema.